In [1]:
import pandas as pd

df=pd.read_csv("/kaggle/input/fake-news/train.csv")


In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().value_counts

<bound method DataFrame.value_counts of           id  title  author   text  label
0      False  False   False  False  False
1      False  False   False  False  False
2      False  False   False  False  False
3      False  False   False  False  False
4      False  False   False  False  False
...      ...    ...     ...    ...    ...
20795  False  False   False  False  False
20796  False  False   False  False  False
20797  False  False   False  False  False
20798  False  False   False  False  False
20799  False  False   False  False  False

[20800 rows x 5 columns]>

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.drop('id', axis=1, inplace=True)
df.drop('author', axis=1, inplace=True)

df.head()[:5]

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.dropna(axis=0,inplace=True)

In [7]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [8]:
df.shape

(20203, 3)

In [9]:
df['label'].value_counts()

label
0    10387
1     9816
Name: count, dtype: int64

In [10]:
import spacy


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
nlp = spacy.load("en_core_web_lg")

In [12]:
counter=0
for text in df['text']:
    if counter <= 5 :
        #print(text)
        continue
      #  print(title)
counter+=1

In [13]:
df.head()

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[======--------------------------------------------] 12.4% 205.7/1662.8MB downloaded

In [ ]:
vector=wv["King"]
vector.shape

In [ ]:
def process_text(text):
  doc=nlp(text)
  flitered_tokens = []
  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    else:
      flitered_tokens.append(token.lemma_)
  return wv.get_mean_vector(flitered_tokens)


In [ ]:
df2=df[:2000]

In [ ]:
df2['text_vector']=df2['text'].apply(lambda text : process_text(text))

In [ ]:
df2.head()

In [ ]:
df2['title_vector']=df2['title'].apply(lambda text : process_text(text))

In [ ]:
y=df2['label']
y

In [ ]:
X=df2.drop(['label','title','text'],axis=1)
X

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler= MinMaxScaler()

In [ ]:
import numpy as np

def scaling_data(df,columns):

  for index, row in df.iterrows():

      word_vectors_array=row[columns]

      scaled_word_vectors = scaler.fit_transform(word_vectors_array.reshape(-1,1))

      single_values = [value[0] for value in scaled_word_vectors]

      row[columns]=np.array(single_values)

      # np.stack(row[columns])

      # row[columns]=(single_values)

  return df


In [ ]:
X=scaling_data(X,"text_vector")
X=scaling_data(X,"title_vector")
X

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(
    X, y, test_size=0.2 ,random_state= 500
)

X_train

In [ ]:
def merge_arrays(row):
    return np.concatenate([row['text_vector'], row['title_vector']])
X_train["concatenate"]=X_train.apply(merge_arrays, axis=1)
X_test["concatenate"]=X_test.apply(merge_arrays, axis=1)
X_train

In [ ]:
# Function to split vectors into separate columns
def split_vectors(row):
    return pd.Series(row['concatenate'])

# Apply the function to create a new DataFrame
X_train = X_train.apply(split_vectors, axis=1)
X_test = X_test.apply(split_vectors, axis=1)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler #fixed import

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = MultinomialNB()

clf.fit(X_train, y_train)


In [ ]:
from sklearn import metrics

y_pred = np.around(clf.predict(X_test))
print(metrics.classification_report(y_test,y_pred))